In [ ]:
!pip install -q scikit-learn transformers matplotlib

In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from concurrent.futures import ProcessPoolExecutor

# Function to process a JSON file and extract the relevant data
def extract_data(file_path, max_length):
    with open(file_path) as file:
        data = json.load(file)
    
    extracted_data = []
    for document in data:
        for turn in document['TURNS']:
            for name in turn['NAMES']:
                utterance = ' '.join(turn['UTTERANCES']).lower()
                if max_length is None or len(utterance) <= max_length:
                    extracted_data.append({
                        'name': name,
                        'utterance': utterance,
                        'turn_number': turn['NUMBER']
                    })
    return extracted_data

# Wrapper function to enable multiprocessing
def extract_data_wrapper(args):
    return extract_data(*args)

# Load data from JSON files in the specified folder and its subdirectories
def load_data(folder_path, max_length=None, num_workers=4):
    all_texts = []
    all_names = []
    file_paths = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.json'):
                file_paths.append(os.path.join(root, file))

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(extract_data_wrapper, [(file_path, max_length) for file_path in file_paths])

    for result in results:
        for item in result:
            all_texts.append(item['utterance'])
            all_names.append(item['name'])

    print(f"Processed {len(file_paths)} files and extracted {len(all_texts)} utterances suitable within length {max_length}.")
    return all_texts, all_names

# Preprocess text data using a tokenizer
def preprocess_texts(texts, tokenizer, max_length=128):
    if not texts:  # Check if texts is empty
        print("No texts available for tokenization.")
        return None, None
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    return encodings.input_ids, encodings.attention_mask

# Define the Autoencoder model using a pretrained BERT model as the encoder
class BertAutoencoder(nn.Module):
    def __init__(self, bert_model_name, lstm_units=256, max_length=128):
        super(BertAutoencoder, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.encoder = self.bert.encoder
        self.decoder = nn.LSTM(self.bert.config.hidden_size, lstm_units, batch_first=True)
        self.output_layer = nn.Linear(lstm_units, self.bert.config.vocab_size)
        self.max_length = max_length

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        encoder_outputs = bert_outputs.last_hidden_state
        decoder_outputs, _ = self.decoder(encoder_outputs)
        output = self.output_layer(decoder_outputs)
        return output

def calculate_accuracy(logits, labels):
    _, predicted = torch.max(logits, -1)
    correct = (predicted == labels).float()
    accuracy = correct.sum() / len(correct)
    return accuracy

# Progressive data increment method
def progressive_training(model, train_dataset, initial_size, increment_ratio, max_epochs, device):
    total_size = len(train_dataset)
    test_size = 0.1
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    metrics = []

    # Separate the test data and ensure it's shuffled
    train_size = int((1 - test_size) * total_size)
    test_size = total_size - train_size
    train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=4)

    current_size = initial_size

    for epoch in range(max_epochs):
        if current_size > train_size:
            break

        current_train_indices = torch.randperm(train_size)[:int(current_size)]
        current_train_dataset = torch.utils.data.Subset(train_dataset, current_train_indices)
        train_loader = DataLoader(current_train_dataset, batch_size=512, shuffle=True, num_workers=4)

        model.train()
        train_loss, train_accuracy = 0, 0
        for inputs, targets, masks in train_loader:
            inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, masks)
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            acc = calculate_accuracy(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_accuracy += acc.item()

        train_accuracy /= len(train_loader)

        model.eval()
        val_loss, val_accuracy = 0, 0
        with torch.no_grad():
            for inputs, targets, masks in test_loader:
                inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)
                outputs = model(inputs, masks)
                loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
                acc = calculate_accuracy(outputs.view(-1, outputs.size(-1)), targets.view(-1))
                val_loss += loss.item()
                val_accuracy += acc.item()

        val_accuracy /= len(test_loader)

        metrics.append((current_size, train_loss/len(train_loader), val_loss/len(test_loader), train_accuracy, val_accuracy))
        current_size += current_size * increment_ratio

        print(f'Epoch {epoch+1}/{max_epochs} | Data size: {current_size} | Train Loss: {train_loss/len(train_loader):.8f} | Val Loss: {val_loss/len(test_loader):.8f} | Train Acc: {train_accuracy:.2f}% | Val Acc: {val_accuracy:.2f}%')
    
    return metrics

# Plotting function
def plot_metrics(metrics):
    sizes, train_losses, val_losses, train_accuracies, val_accuracies = zip(*metrics)

    plt.figure(figsize=(12, 8))

    plt.subplot(2, 1, 1)
    plt.plot(sizes, train_losses, label='Training Loss')
    plt.plot(sizes, val_losses, label='Validation Loss')
    plt.xlabel('Data Size')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss vs Data Size')

    plt.subplot(2, 1, 2)
    plt.plot(sizes, train_accuracies, label='Training Accuracy')
    plt.plot(sizes, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Data Size')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy vs Data Size')

    plt.tight_layout()
    plt.show()

# Main function
def main(folder_path, max_text_length, initial_size, increment_ratio, max_epochs, num_workers=4):
    texts, names = load_data(folder_path, max_text_length, num_workers)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    input_ids, attention_masks = preprocess_texts(texts, tokenizer, max_length=128)
    if input_ids is None:  # Check if tokenization failed
        return

    dataset = TensorDataset(input_ids, input_ids, attention_masks)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BertAutoencoder('bert-base-uncased').to(device)

    metrics = progressive_training(model, dataset, initial_size, increment_ratio, max_epochs, device)

    # Save the model
    torch.save(model.state_dict(), 'trained_autoencoder.pth')

    plot_metrics(metrics)

if __name__ == "__main__":
    folder_path = '/workspace/slice-monorepo/cl_cr3/aligneddata/c=3'  # Update with your actual data path
    max_text_length = 128  # Set the maximum text length for BERT
    initial_size = 1024  # Initial training data size
    increment_ratio = 0.1  # Increment ratio for progressive training
    max_epochs = 50000  # Maximum number of epochs
    num_workers = 4  # Number of workers for data loading and preprocessing
    main(folder_path, max_text_length, initial_size, increment_ratio, max_epochs, num_workers)


Processed 330 files and extracted 762187 utterances suitable within length 128.
Epoch 1/50000 | Data size: 1126.4 | Train Loss: 10.29248619 | Val Loss: 10.13624070 | Train Acc: 0.00% | Val Acc: 0.04%
Epoch 2/50000 | Data size: 1239.0400000000002 | Train Loss: 10.06482665 | Val Loss: 9.87027234 | Train Acc: 0.05% | Val Acc: 0.06%
Epoch 3/50000 | Data size: 1362.9440000000002 | Train Loss: 9.82792568 | Val Loss: 9.63186256 | Train Acc: 0.06% | Val Acc: 0.07%
Epoch 4/50000 | Data size: 1499.2384000000002 | Train Loss: 9.59657923 | Val Loss: 9.39984893 | Train Acc: 0.07% | Val Acc: 0.08%
Epoch 5/50000 | Data size: 1649.16224 | Train Loss: 9.38470745 | Val Loss: 9.17551332 | Train Acc: 0.08% | Val Acc: 0.08%
Epoch 6/50000 | Data size: 1814.0784640000002 | Train Loss: 9.12226534 | Val Loss: 8.87898526 | Train Acc: 0.08% | Val Acc: 0.09%
Epoch 7/50000 | Data size: 1995.4863104 | Train Loss: 8.83133721 | Val Loss: 8.58319162 | Train Acc: 0.09% | Val Acc: 0.10%
Epoch 8/50000 | Data size: 2195.0